In [34]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests


In [19]:
def get_it(url, title):
    final_url = 'https://en.wikipedia.org'+url
    r = requests.get(final_url)
    soup = bs(r.content)
    
    table = soup.find("table", attrs={"class":"infobox vevent"})
    try:
        headers = table.find_all('th', attrs={"scope":"row"})
    except:
        print(soup.prettify())
    
    d = {}
    d['Title'] = title
    rows = table.find_all('tr')
    for row in rows:
        header = row.find('th',attrs={"scope":"row"})
        if header:
            key = header.get_text("").strip()
            value = row.find('td').get_text().strip().replace('\xa0','').split('\n')
            d[key] = value
    return d

In [20]:
url = "https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films"
r = requests.get(url)
soup = bs(r.content)

tables = soup.find_all("table",attrs = {"class":"wikitable sortable"})[:-1]
lst = []
for table in tables:
    films = table.find_all('i')
    for i in films:
        if i.a:
            title = i.a.get_text(strip=True)
            link = i.a['href']
            if link == '/wiki/True-Life_Adventures':
                continue
            d = get_it(link,title)
            lst.append(d)
        else:
            link: "No Link"

In [27]:
import json
with open('wiki_movs.json','w',encoding ='utf-8') as f:
    json.dump(lst,f,ensure_ascii=False,indent=6)

In [28]:
def load_json(title):
    with open(title,encoding = 'utf-8') as f:
        return json.load(f)

In [29]:
mov_info = load_json('wiki_movs.json')

In [30]:
import urllib
def get_scores(title):
    base_url = 'http://www.omdbapi.com/?'
    parameters = {'apikey':'2c848dbe','t':title}
    end_url = urllib.parse.urlencode(parameters)
    final_url = base_url + end_url
    return requests.get(final_url).json()

def get_rts(temp):
    r = temp.get('Ratings',[])
    for ele in r:
        if ele['Source'] == 'Rotten Tomatoes':
            return ele['Value']
        return None

In [31]:
for mov in mov_info:
    title = mov['Title']
    temp = get_scores(title)
    mov['Metascore'] = temp.get('Metascore', None)
    mov['imdbRating'] = temp.get('imbdRating', None)
    mov['Rotten_Tomatoes'] = get_rts(temp)
    mov['Genre'] = temp.get('Genre', None)
    mov['Rated'] = temp.get('Rated', None)
    mov['BoxOffice_Api'] = temp.get('BoxOffice', None)
    mov['Type'] = temp.get('Type',None)

In [32]:
import json
with open('Final_data.json','w',encoding = 'utf-8') as f:
    json.dump(mov_info,f,ensure_ascii=False,indent=6)

In [35]:
df = pd.DataFrame(mov_info)

In [36]:
df.to_csv('Final_Data.csv', index = False)

In [39]:
df

,Title,Productioncompany,Release date,Running time,Country,Language,Box office,Metascore,imdbRating,Rotten_Tomatoes,...,Executive producer,Producers,Editors,Distributor,Japanese,Hepburn,Adaptation by,Animation by,Traditional,Simplified
0,Academy Award Review of Walt Disney Cartoons,[Walt Disney Productions],"[May19,1937(1937-05-19)]",[41 minutes (74 minutes 1966 release)],[United States],[English],[$45.472],N/A,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Snow White and the Seven Dwarfs,[Walt Disney Productions],"[December21,1937(1937-12-21) (Carthay Circle T...",[83 minutes],[United States],[English],[$418 million[2]],95,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pinocchio,[Walt Disney Productions],"[February7,1940(1940-02-07) (Center Theatre)[1...",[88 minutes],[United States],[English],[$164million],99,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fantasia,[Walt Disney Productions],"[November13,1940(1940-11-13)]",[126 minutes[1]],[United States],[English],[$76.4–$83.3 million (United States and Canada...,96,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,The Reluctant Dragon,[Walt Disney Productions],"[June27,1941(1941-06-27)[1]]",[74 minutes],[United States],[English],"[$960,000 (worldwide rentals) [3]]",N/A,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,Flora & Ulysses,NaN,"[February19,2021(2021-02-19)]",[95 minutes],[United States],[English],NaN,62,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
438,Raya and the Last Dragon,NaN,"[March5,2021(2021-03-05) (United States)]",[107 minutes[5]],[United States],[English],[$122.7 million],75,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
439,Cruella,NaN,"[May18,2021(2021-05-18) (El Capitan Theatre), ...",[134 minutes[4]],[United States],[English],[$226.2 million[8][9]],59,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440,Luca,NaN,"[June13,2021(2021-06-13) (Aquarium of Genoa), ...",[95 minutes],[United States],[English],[$27.8 million[1][2]],71,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
